# Multiple Regression (Interpretation) on House sales data

### Fire up graphlab create

In [2]:
import graphlab

### Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\agraw\AppData\Local\Temp\graphlab_server_1504899806.log.0


This non-commercial license of GraphLab Create for academic use is assigned to agrawal.pr@husky.neu.edu and will expire on March 12, 2018.


### Explore house sales data

In [4]:
sales[0:1]

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 1

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|       0       |   1955   |      0       |  98178  | 47.51123398 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
+---------------+---------------+-----+
[1 rows x 21 columns]

### Split the data

In [5]:
train_data, test_data = sales.random_split(.8,seed=0)

### Learning a multiple regression model

### Example Model

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.032561     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Extracting the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



### Making Predictions

In [7]:
example_predictions = example_model.predict(train_data)
print example_predictions[0]

271789.505878


### Compute RSS

RSS function

In [8]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)

    # Then compute the residuals/errors
    residuals = outcome - predictions

    # Then square and add them up
    residuals_squared = residuals*residuals
    RSS = residuals_squared.sum()

    return(RSS)    

### Run example model on test data

In [12]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print "The RSS for example model: " +str(rss_example_train)

The RSS for example model: 2.7376153833e+14


### Create new features

Transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

In [24]:
from math import log

* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 

In [13]:
# create the bedroom squared new feature in both TEST and TRAIN data
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [21]:
# create the bathroom bedroom new feature in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data.apply(lambda x: x['bedrooms']*x['bathrooms'])
test_data['bed_bath_rooms'] = test_data.apply(lambda x: x['bedrooms']*x['bathrooms'])

In [26]:
# create the log of sqft_living new feature in both TEST and TRAIN data
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

In [28]:
# create the lat long new feature in both TEST and TRAIN data
train_data['lat_plus_long'] = train_data.apply(lambda x: x['lat'] + x['long'])
test_data['lat_plus_long'] = test_data.apply(lambda x: x['lat'] + x['long'])

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms is an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will see why later.

In [31]:
# checking new features
train_data[0:1]

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float
	bedrooms_squared	float
	bed_bath_rooms	float
	log_sqft_living	float
	lat_plus_long	float

Rows: 1

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    1180.0   |   5650   |   1    |     0      |  0   |     3     |   7   |    1180    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|       0       |   1955   |      0       |  98178  | 47.51123398 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
+---------------+---------------+-----+
[1 rows x 25 columns]

 **What is the Mean (arithmetic average) values of 4 new features on TEST data?

In [32]:
test_data['bedrooms_squared'].mean()

12.44667770158429

In [33]:
test_data['bed_bath_rooms'].mean()

7.503901631591395

In [34]:
test_data['log_sqft_living'].mean()

7.55027467964594

In [35]:
test_data['lat_plus_long'].mean()

-74.65333497217306

### Learning Multiple Models

* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [36]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

### Model 1

In [38]:
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)
model_1_summary = model_1.get('coefficients')

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.019049     | 4074878.213096     | 236378.596455 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

### Model 2


In [40]:
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)
model_2_summary = model_2.get('coefficients')

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.019052     | 4014170.932927     | 235190.935428 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

### Model 3

In [41]:
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)
model_3_summary = model_3.get('coefficients')

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 9

Number of unpacked features : 9

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.015042     | 3193229.177894     | 228200.043155 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

### Weights of each of the models

In [42]:
model_1_summary

name,index,value,stderr
(intercept),None,-56140675.7444,1649985.42028
sqft_living,None,310.263325778,3.18882960408
bedrooms,None,-59577.1160682,2487.27977322
bathrooms,None,13811.8405418,3593.54213297
lat,None,629865.789485,13120.7100323
long,None,-214790.285186,13284.2851607


In [43]:
model_2_summary

name,index,value,stderr
(intercept),None,-54410676.1152,1650405.16541
sqft_living,None,304.449298057,3.20217535637
bedrooms,None,-116366.043231,4805.54966546
bathrooms,None,-77972.3305135,7565.05991091
lat,None,625433.834953,13058.3530972
long,None,-203958.60296,13268.1283711
bed_bath_rooms,None,26961.6249092,1956.36561555


In [44]:
model_3_summary

name,index,value,stderr
(intercept),None,-52974974.0602,1615194.9439
sqft_living,None,529.196420564,7.69913498511
bedrooms,None,28948.5277313,9395.72889106
bathrooms,None,65661.207231,10795.3380703
lat,None,704762.148408,1292011141.66
long,None,-137780.01994,1292011141.57
bed_bath_rooms,None,-8478.36410518,2858.95391257
bedrooms_squared,None,-6072.38466067,1494.97042777
log_sqft_living,None,-563467.784269,17567.8230814
lat_plus_long,None,-83217.1979248,1292011141.58


### Comparing multiple models


### RSS of train data using all three models

In [45]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_train_model_1 = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
rss_train_model_2 = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
rss_train_model_3 = get_residual_sum_of_squares(model_3, train_data, train_data['price'])

**Which model (1, 2 or 3) has lowest RSS on TRAINING Data?**

In [47]:
print "RSS of train data using model 1: " + str(rss_train_model_1)

RSS of train data using model 1: 9.71328233544e+14


In [48]:
print "RSS of train data using model 2: " + str(rss_train_model_2)

RSS of train data using model 2: 9.61592067856e+14


In [49]:
print "RSS of train data using model 3: " + str(rss_train_model_3)

RSS of train data using model 3: 9.05276314555e+14


### So RSS of the model 3 is lowest on the train data

### RSS of test data using all three models

In [50]:
# Compute the RSS on TEST data for each of the three models and record the values:
rss_test_model_1 = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
rss_test_model_2 = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
rss_test_model_3 = get_residual_sum_of_squares(model_3, test_data, test_data['price'])

**Which model (1, 2 or 3) has lowest RSS on TEST Data?**

In [51]:
print "RSS of test data using model 1: " + str(rss_test_model_1)

RSS of test data using model 1: 2.26568089093e+14


In [53]:
print "RSS of test data using model 2: " + str(rss_test_model_2)

RSS of test data using model 2: 2.24368799994e+14


In [54]:
print "RSS of test data using model 3: " + str(rss_test_model_3)

RSS of test data using model 3: 2.51829318952e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

### So RSS of the model 2 is lowest on the test data

# Hence model 2 is the optimal model